In [ ]:

%pip install python-dotenv

In [16]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# print(os.environ)

# os.environ["OPENAI_API_TYPE"] = os.environ.get('OPENAI_API_TYPE')
# os.environ["OPENAI_API_BASE"] = os.environ.get('OPENAI_API_BASE')
# os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY')
# os.environ["OPENAI_API_VERSION"] = os.environ.get('OPENAI_API_VERSION')


AI_TEMPERATURE=0.1
AI_MAX_TOKENS=2048

In [17]:
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI

def load_azureLLM():
    # chat
    chat = AzureChatOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_CHAT'),
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    # llm
    llm = AzureOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_LLM'),
        model_name="text-davinci-003",
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    
    return llm, chat

llm, chat = load_azureLLM()
# print(llm)
# print(chat)
# llm("Tell me a joke")

## 营养建议

In [18]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

goal = "减脂瘦身"
fitness = """{
    "frequency": "3",
    "date": "周一、周三、周六",
    "time": "晚上",
    "equipment": "瑜伽垫",
    "music": "舒缓类",
    "exclude": "跑跳类动作",
    "duration": "不超过20分钟"
}"""
person = """{
    "gender": "女",
    "age": "20岁",
    "height": "150cm",
    "weight": "50kg"
}"""
health = """{
    "health": "良好",
    "drug": "无",
    "allergy": "牛奶",
}"""
foods = """{
    "frequency": "2",
    "favorite ": "香蕉",
    "dislike ": "青椒",
    "allergy": "牛奶"
    "snacks": "50kg"
}"""



prompt_string = """你将作为一位备受赞誉的健康与营养专家 FitnessGPT。\
    我希望你能根据我提供的个人信息、健康状态、健身目标、运动偏好及饮食偏好，为我定制一套个性化的饮食和运动计划。\
    我需要你为我详细制定运动计划。需要列出每次运动课程的动作清单。课程动作需要包括运动时间或轮次细节。\
    我也需要你帮我详细规划我的饮食计划，并列出一份详细的饮食清单，清单上需要包括每种食品的数量及饮食方式。\
    请避免任何不必要的描述性文本，以及解释性描述，只输出规划设计的计划内容。不论在什么情况下，都请保持角色设定不变。\
    最后，我希望你能给我列出7条运动健身场景的励志箴言，帮助我保持对目标的激励。\
    Let's think step by step.\
    
    我的个人信息: ```{private_info}```。\
    我的健康状态是: ```{health_info}```。\
    我的健身目标是: ```{fitness_goal}```。\
    我的运动偏好是:```{fitness_preference}```。\
    我的饮食偏好是: ```{fitness_foods}```。

    """


prompt = PromptTemplate(
    input_variables=["private_info","health_info","fitness_goal","fitness_preference","fitness_foods"],
    template=prompt_string
)

chain = LLMChain(llm=llm,prompt=prompt)

print(chain.run(
    private_info=person,
    health_info=health,
    fitness_goal=goal,
    fitness_preference=fitness,
    fitness_foods=foods))


运动计划：
1. 周一：
    - 时间：20分钟
    - 动作：
        - 俯卧撑：3组，每组10个
        - 仰卧起坐：3组，每组10个
        - 俯身支撑：3组，每组10个
        - 侧身支撑：3组，每组10个
        - 卷腹：3组，每组10个
        - 平板支撑：3组，每组10个
2. 周三：
    - 时间：20分钟
    - 动作：
        - 弓箭步：3组，每组10个
        - 深蹲：3组，每组10个
        - 卷腹：3组，每组10个
        - 俯卧撑：3组，每组10个
        - 仰卧起坐：3组，每组10个
        - 俯身支撑：3组，每组10个
3. 周六：
    - 时间：20分钟
    - 动作：
        - 弓箭步：3组，每组10个
        - 深蹲：3组，每组10个
        - 卷腹：3组，每组10个
        - 俯卧撑：3组，每组10个
        - 仰卧起坐：3组，每组10个
        - 俯身支撑：3组，每组10个

饮食计划：
- 早餐：
    - 燕麦片：50克
    - 鸡蛋：1个
    - 水果：1个
- 午餐：
    - 米饭：100克
    - 鸡肉：50克
    - 蔬菜：100克
- 晚餐：
    - 燕麦片：50克
    - 鸡蛋：1个
    - 水果：1个
- 加餐：
    - 香蕉：1个

7条运动健身场景的励志箴言：
1. 健身不是一时的冲动，而是一种长期的习惯。
2. 健身不是为了改变外表，而是为了改变内心。
3. 健身不是为了赢得别人的赞赏，而是为了赢得自己的尊重。
4. 健身不是为了追求完美，而是为了追求健康。
5. 健身不是为了比谁更强壮，而是为了比谁更健康。
6. 健身不是为了拼搏，而是为了拼活力。
7. 健身不是为了拼搏，而是为了拼活力。


## 评估动作

In [19]:
from langchain.prompts import ChatPromptTemplate

prompt_evaluate_action_standards = """评估维度：\
1. 力量：指肌肉在运动中产生的最大力量。常见的力量训练包括举重、俯卧撑、深蹲等。通过计次评估。\
2. 速度：指肌肉完成一定距离或动作所需时间的能力。常见的速度训练包括快速小跑、跳绳等。通过计时评估。\
3. 耐力：指肌肉长时间持续工作的能力。常见的耐力训练包括深蹲跳、高抬腿等。通过计次或计时评估。\
4. 灵敏：指肌肉对刺激的快速反应能力。常见的灵敏度训练包括跳绳、跳跃、击剑等。通过计次评估。\
5. 协调：指身体各部位肌肉协调运动的能力，常见的协调性训练包括瑜伽、桑巴、舞蹈等。通过计时评估。\
6. 稳定：指骨骼及肌肉在身体静止或运动状态下维持平衡的能力。常见的稳定性训练包括平板支撑、单脚站立等。通过计时评估。\
评估方法：\
根据动作的计次和计时数据，严格评估动作在每个维度的得分。\
评估得分，最低0分，最高10分。如果无法评估，则为0分。
"""

person_info = """{
    "gender": "女",
    "age": "20岁",
    "height": "150cm",
    "weight": "50kg"
}"""
fitness_data = """{
    "action": "深蹲",
    "timing": "60秒",
    "count_type": "次数",
    "count_result": "30次"
}"""

prompt_evaluate_action = """
您是资深的健身运动专家，请严格以``` {standards} ```为评估标准，对用户的运动数据进行评估。\
请仅对用户的运动数据进行评估，不要额外评价。\
严格以json格式化输出下列字段:\
dimension:
    score:
    reason:

Let's think step by step.\

运动数据：### {data} ###
"""


evaluate_template = ChatPromptTemplate.from_template(prompt_evaluate_action)

evaluate_messages = evaluate_template.format_messages(
                    standards=prompt_evaluate_action_standards,
                    data=fitness_data,
                    )

action_evaluate = chat(evaluate_messages)
print(action_evaluate.content)

{
    "dimension": "力量",
    "score": 8,
    "reason": "完成了30次深蹲，表现出较强的肌肉力量"
}

{
    "dimension": "耐力",
    "score": 7,
    "reason": "在60秒内完成了30次深蹲，表现出较好的肌肉耐力"
}

{
    "dimension": "稳定",
    "score": 6,
    "reason": "完成30次深蹲，但在60秒内未能保持完美的平衡，需要加强稳定性训练"
}

{
    "dimension": "速度",
    "score": 0,
    "reason": "数据中未提供完成一定距离或动作所需时间的信息，无法评估速度维度得分"
}

{
    "dimension": "灵敏",
    "score": 0,
    "reason": "数据中未提供灵敏度训练的信息，无法评估灵敏维度得分"
}

{
    "dimension": "协调",
    "score": 0,
    "reason": "数据中未提供协调性训练的信息，无法评估协调维度得分"
}


## 动作升降阶

## 修正课程

## 课程生成